토큰화  
- 주어진 텍스트를 단어 또는 문자 단위로 자르는 것

In [1]:
en_text = "A Dog Run back corner near spare bedrooms"

import nltk
nltk.download('punkt')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


True

In [2]:
from nltk.tokenize import word_tokenize
print(word_tokenize(en_text))

['A', 'Dog', 'Run', 'back', 'corner', 'near', 'spare', 'bedrooms']


In [3]:
# 한국어는 단순 띄어쓰기 단위로 나누면 같은 단어가 다른 단어로 인식되어 단어 집합 크기가
# 불필요하게 커짐
# 단어집합 : 중복을 제거한 텍스트의 총 단어의 집합(set)
print(en_text.split())

['A', 'Dog', 'Run', 'back', 'corner', 'near', 'spare', 'bedrooms']


토치텍스트(Torchtext)  
텍스트에 대한 여러 추상화 기능을 제공하는 자연어 처리 라이브러리  

제공 기능  
- 파일 로드하기(File Loading) : 다양한 포맷의 코퍼스 로드
- 토큰화(Tokenization) : 문장을 단어 단위로 분리
- 단어 집합(Vocab) : 단어 집합 생성
- 정수 인코딩(Integer encoding) : 전체 코퍼스의 단어들을 각각의 고유한 정수로 맵핑
- 단어 벡터(Word Vector) : 단어 집합의 단어들에 고유한 임베딩 벡터를 만들어줌. 랜덤값으로 초기화한 값일 수도 있고, 사전 훈련된 임베딩 벡터들을 로드할 수 있음
- 배치화(Batching) : 훈련 샘플들의 배치를 만들어줌. 이 과정에서 패딩 작업(Padding)도 이뤄짐

In [4]:
import urllib.request
import pandas as pd
urllib.request.urlretrieve(
    "https://raw.githubusercontent.com/LawrenceDuan/IMDb-Review-Analysis/master/IMDb_Reviews.csv",
    filename="IMDb_Reviews.csv")

('IMDb_Reviews.csv', <http.client.HTTPMessage at 0x7fedc0316640>)

In [5]:
df = pd.read_csv('IMDb_Reviews.csv', encoding='latin1')
df.head()

,review,sentiment
0,My family and I normally do not watch local mo...,1
1,"Believe it or not, this was at one time the wo...",0
2,"After some internet surfing, I found the ""Home...",0
3,One of the most unheralded great works of anim...,1
4,"It was the Sixties, and anyone with long hair ...",0


In [6]:
# 전체 샘플의 개수
print('total sample:', len(df))

total sample: 50000


In [7]:
# 25,000개씩 분리
train_df = df[:25000]
test_df = df[25000:]
train_df.to_csv('train_data.csv', index=False)
test_df.to_csv('test_data.csv', index=False)

In [8]:
# 필드 정의하기(torchtext.data)

from torchtext.legacy.data import TabularDataset
from torchtext.legacy import data

# sequential : 시퀀스 데이터 여부 (True가 기본값)
# use_vocab : 단어 집합을 만들 것인지 여부 (True가 기본값)
# tokenize : 어떤 토큰화 함수를 사용할 것인지 지정 (string.split이 기본값)
# lower : 영어 데이터를 전부 소문자화함 (False가 기본값)
# batch_first : 미니 배치 차원을 맨 앞으로 하여 데이터를 불러올 것인지 여부 (False가 기본값)
# is_target : 레이블 데이터 여부 (False가 기본값)
# fix_length : 최대 허용 길이. 이 길이에 맞춰서 패딩 작업(Padding)이 진행됨

# 텍스트 객체
TEXT = data.Field(
    sequential=True, use_vocab=True, tokenize=str.split, 
    lower=True, batch_first=True, fix_length=20)

# 레이블 객체
LABEL = data.Field(
    sequential=False, use_vocab=False, batch_first=False, is_target=True)

In [9]:
# 데이터셋 만들기

from torchtext.legacy.data import TabularDataset

# path : 파일이 위치한 경로.
# format : 데이터의 포맷.
# fields : 위에서 정의한 필드를 지정. 
# 첫번째 원소는 데이터 셋 내에서 해당 필드를 호칭할 이름, 두번째 원소는 지정할 필드.
# skip_header : 데이터의 첫번째 줄은 무시
train_data, test_data = TabularDataset.splits(
    path='.', train='train_data.csv', test='test_data.csv', format='csv',
    fields=[('text', TEXT), ('label', LABEL)], skip_header=True)
print('train : ', len(train_data))
print('test : ', len(test_data))
print(vars(train_data[0]))

train :  25000
test :  25000
{'text': ['my', 'family', 'and', 'i', 'normally', 'do', 'not', 'watch', 'local', 'movies', 'for', 'the', 'simple', 'reason', 'that', 'they', 'are', 'poorly', 'made,', 'they', 'lack', 'the', 'depth,', 'and', 'just', 'not', 'worth', 'our', 'time.<br', '/><br', '/>the', 'trailer', 'of', '"nasaan', 'ka', 'man"', 'caught', 'my', 'attention,', 'my', 'daughter', 'in', "law's", 'and', "daughter's", 'so', 'we', 'took', 'time', 'out', 'to', 'watch', 'it', 'this', 'afternoon.', 'the', 'movie', 'exceeded', 'our', 'expectations.', 'the', 'cinematography', 'was', 'very', 'good,', 'the', 'story', 'beautiful', 'and', 'the', 'acting', 'awesome.', 'jericho', 'rosales', 'was', 'really', 'very', 'good,', "so's", 'claudine', 'barretto.', 'the', 'fact', 'that', 'i', 'despised', 'diether', 'ocampo', 'proves', 'he', 'was', 'effective', 'at', 'his', 'role.', 'i', 'have', 'never', 'been', 'this', 'touched,', 'moved', 'and', 'affected', 'by', 'a', 'local', 'movie', 'before.', 'imagin

In [10]:
print(vars(train_data[0]))

{'text': ['my', 'family', 'and', 'i', 'normally', 'do', 'not', 'watch', 'local', 'movies', 'for', 'the', 'simple', 'reason', 'that', 'they', 'are', 'poorly', 'made,', 'they', 'lack', 'the', 'depth,', 'and', 'just', 'not', 'worth', 'our', 'time.<br', '/><br', '/>the', 'trailer', 'of', '"nasaan', 'ka', 'man"', 'caught', 'my', 'attention,', 'my', 'daughter', 'in', "law's", 'and', "daughter's", 'so', 'we', 'took', 'time', 'out', 'to', 'watch', 'it', 'this', 'afternoon.', 'the', 'movie', 'exceeded', 'our', 'expectations.', 'the', 'cinematography', 'was', 'very', 'good,', 'the', 'story', 'beautiful', 'and', 'the', 'acting', 'awesome.', 'jericho', 'rosales', 'was', 'really', 'very', 'good,', "so's", 'claudine', 'barretto.', 'the', 'fact', 'that', 'i', 'despised', 'diether', 'ocampo', 'proves', 'he', 'was', 'effective', 'at', 'his', 'role.', 'i', 'have', 'never', 'been', 'this', 'touched,', 'moved', 'and', 'affected', 'by', 'a', 'local', 'movie', 'before.', 'imagine', 'a', 'cynic', 'like', 'me

In [11]:
# 필드 구성 확인.
print(train_data.fields.items())

dict_items([('text', <torchtext.legacy.data.field.Field object at 0x7fedc245beb0>), ('label', <torchtext.legacy.data.field.Field object at 0x7fedc245bb20>)])


In [12]:
# 단어 집합(Vocabulary) 만들기

# min_freq : 단어 집합에 추가 시 단어의 최소 등장 빈도 조건을 추가.
# max_size : 단어 집합의 최대 크기를 지정.
TEXT.build_vocab(train_data, min_freq=10, max_size=10000)

print('단어 집합의 크기:', len(TEXT.vocab))

# 단어 집합의 크기를 10,000개로 제한하였지만 실제 생성된 단어 집합의 크기는 0번 단어부터 
# 10,001번 단어까지 총 10,002개. 
# 이는 토치텍스트가 임의로 특별 토큰인 <unk>와 <pad>를 추가하였기 때문.
# 토치텍스트는 <unk>의 번호는 0번, <pad>의 번호는 1번을 부여함
# <unk>는 단어 집합에 없는 단어를 표현할 때 사용되며, 
# <pad>는 길이를 맞추는 패딩 작업을 할 때 사용됨.
# print(TEXT.vocab.stoi)

단어 집합의 크기: 10002


In [13]:
# 데이터로더 만들기
from torchtext.legacy.data import Iterator

batch_size = 5
train_loader = Iterator(dataset=train_data, batch_size=batch_size)
test_loader = Iterator(dataset=test_data, batch_size=batch_size)

print('train data mini batch size : ', len(train_loader))
print('test data mini batch size : ', len(test_loader))

batch = next(iter(train_loader)) # 첫번째 미니배치
print('type(batch) : ', type(batch))
print('batch.text : ', batch.text)

train data mini batch size :  5000
test data mini batch size :  5000
type(batch) :  <class 'torchtext.legacy.data.batch.Batch'>
batch.text :  tensor([[   0,    7,    3,   49,  838,   20,    6,    0,   43,    0,    6,   98,
            5,   11,   15,  386,    0,    5,    2,  501],
        [3013,    0,   10,  150, 2144, 1444,  354,   32,  949,    6,    2,  501,
            5,  104, 8700, 6905,    4,    0,   35,  224],
        [   9,  205,  141,   12,  273,   55, 2236,  147,  171, 2714,    5,    3,
          167,    5,    2,  125,  186,    0,    2, 1849],
        [  10,   20,    7, 1190, 2440,   81,  219,    0, 1303,   17, 2611,   30,
            4,  276,  195,   19,    3,  422, 1834,  143],
        [  10,   25, 6137,    5,  366,  909,    4, 2654,  390, 2167, 4653,    0,
            7,    3, 3739,  852, 1391,   16, 2035,    0]])


배치 크기가 5이기 때문에 5개의 샘플이 출력됨. 초기에 Field를 정의할 때 fix_length를 20으로 지정 했기때문에 하나의 미니 배치 크기는 20 (배치 크기 * fix_length).  

각 샘플의 중간 중간에 숫자 0은 unk 토큰 번호로 단어 집합에 포함되지 못한 단어임.

pad토큰이 사용되는 경우
pad 토큰이 어떤 경우에 사용되는지 보겠습니다. 맨 처음 필드를 정의할 때 fix_length를 20이 아니라 150으로 정의하고, 이후에는 데이터로더를 정의하기까지 모든 실습을 동일하게 진행했다고 가정해봅시다. 그리고 첫번째 미니배치의 첫번째 샘플을 출력해봅시다.

In [14]:
TEXT = data.Field(
    sequential=True, use_vocab=True, tokenize=str.split, 
    lower=True, batch_first=True, fix_length=150)

# 레이블 객체
LABEL = data.Field(
    sequential=False, use_vocab=False, batch_first=False, is_target=True)

train_data, test_data = TabularDataset.splits(
    path='.', train='train_data.csv', test='test_data.csv', format='csv',
    fields=[('text', TEXT), ('label', LABEL)], skip_header=True)
TEXT.build_vocab(train_data, min_freq=10, max_size=10000)
batch_size = 5
train_loader = Iterator(dataset=train_data, batch_size=batch_size)

batch = next(iter(train_loader)) # 첫번째 미니배치
print(batch.text[0]) # 첫번째 미니배치 중 첫번째 샘플

tensor([   0,    7,    3,   49,  838,   20,    6,    0,   43,    0,    6,   98,
           5,   11,   15,  386,    0,    5,    2,  501,    5,    2,  104, 1324,
         819,    2,  236,   12,  123,  121,   23, 1075,    5,    3, 1166, 1486,
         282,    8,   32,    0,  752,    5,    2,  759,  985,    4,   12,   30,
           5,  116,    7,    3, 1352,  114,   22,    0,  116,   34,  248,  475,
          37,   92,   80, 1066,    4,   10,    7,   30,    5,    2, 1806,  106,
          10,   20,    7,  283,    6,  237,  395,  186,  124,  511, 2371,   11,
         142,   73, 4013,   63,    8,    2,  236,   12,   30,    5,    2,  256,
         121,    7,    3, 9436,   43,   39,  140,   12,   41,    6,   28, 3066,
          17,   37,   92,   80, 3481,    2,  111,    2,  121, 7647,    7, 1265,
           0,   52,   23, 2830, 7081,    4, 4633,   12,   23,  950,    6,  166,
         761,   12,   39,   83, 1113,   65,  837,    2,   25,   17,    3, 1420,
         325,    5,   22,  599,  504,  1

기존 샘플의 길이가 150이 되지 않았기 때문에 뒷 부분이 <pad> 번호인 1로 채워짐. 이처럼 서로 다른 길이의 샘플을 동일한 길이로 맞춰주는 작업을 패딩 작업이라고 함

토치텍스트 튜토리얼(Torchtext tutorial) - 한국어

In [15]:
# 네이버 영화 리뷰 데이터 다운로드

import urllib.request
import pandas as pd

urllib.request.urlretrieve("https://raw.githubusercontent.com/e9t/nsmc/master/ratings_train.txt", filename="ratings_train.txt")
urllib.request.urlretrieve("https://raw.githubusercontent.com/e9t/nsmc/master/ratings_test.txt", filename="ratings_test.txt")

train_df = pd.read_table('ratings_train.txt')
test_df = pd.read_table('ratings_test.txt')


In [16]:
train_df.head()

,id,document,label
0,9976970,아 더빙.. 진짜 짜증나네요 목소리,0
1,3819312,흠...포스터보고 초딩영화줄....오버연기조차 가볍지 않구나,1
2,10265843,너무재밓었다그래서보는것을추천한다,0
3,9045019,교도소 이야기구먼 ..솔직히 재미는 없다..평점 조정,0
4,6483659,사이몬페그의 익살스런 연기가 돋보였던 영화!스파이더맨에서 늙어보이기만 했던 커스틴 ...,1


In [17]:
print('train sample : ', len(train_df))
print('test sample : ', len(test_df))

train sample :  150000
test sample :  50000


In [18]:
# 필드 정의하기(torchtext.data)
from torchtext.legacy import data
from konlpy.tag import Mecab

# Mecab을 토크나이저로 사용
tokenizer = Mecab()

# 필드 정의
ID = data.Field(sequential=False, use_vocab=False)
TEXT = data.Field(sequential=True, use_vocab=True, tokenize=tokenizer.morphs,
                  lower=True, batch_first=True, fix_length=20)
LABEL = data.Field(sequential=False, use_vocab=False, is_target=True)

In [19]:
# 데이터셋 만들기
from torchtext.legacy.data import TabularDataset

# 데이터 로드 및 토큰화
train_data, test_data = TabularDataset.splits(
    path='.', train='ratings_train.txt', test='ratings_test.txt', format='tsv',
    fields=[('id', ID), ('text', TEXT), ('label', LABEL)], skip_header=True)

print('train sample : ', len(train_data))
print('test sample : ', len(test_data))
print('vars(train_data[0]) : ', vars(train_data[0]))

train sample :  150000
test sample :  50000
vars(train_data[0]) :  {'id': '9976970', 'text': ['아', '더', '빙', '.', '.', '진짜', '짜증', '나', '네요', '목소리'], 'label': '0'}


In [20]:
# 단어 집합(Vocabulary) 만들기

# min_freq : 단어 집합에 추가 시 단어의 최소 등장 빈도 조건을 추가.
# max_size : 단어 집합의 최대 크기를 지정.
TEXT.build_vocab(train_data, min_freq=10, max_size=10000)
print('단어 집합의 크기 : ', len(TEXT.vocab))
# print(TEXT.vocab.stoi)

단어 집합의 크기 :  10002


In [22]:
from torchtext.legacy.data import Iterator
batch_size = 5
train_loader = Iterator(dataset=train_data, batch_size = batch_size)
test_loader = Iterator(dataset=test_data, batch_size = batch_size)

print('훈련 데이터의 미니 배치 수 : {}'.format(len(train_loader)))
print('테스트 데이터의 미니 배치 수 : {}'.format(len(test_loader)))

batch = next(iter(train_loader)) # 첫번째 미니배치
print('batch.text : ', batch.text)

훈련 데이터의 미니 배치 수 : 30000
테스트 데이터의 미니 배치 수 : 10000
batch.text :  tensor([[ 232,   11, 6338,    4,    5,    1,    1,    1,    1,    1,    1,    1,
            1,    1,    1,    1,    1,    1,    1,    1],
        [   3,    5,   23,   15,  126,    3,    5,  235,   10,  433,   12, 3481,
           30,   25,  325,  682,   86, 2454, 8140,   40],
        [   5,   15,   31,  240,  273,   37, 1294,    3,  388,   11,   53,   55,
            6,    2,    1,    1,    1,    1,    1,    1],
        [4094, 2004,    9,   35,    8,    7,    0,   14, 2038,    4, 3043,   34,
           63, 2164,    1,    1,    1,    1,    1,    1],
        [   0,   39,    0,    5,   23,    0,  146,    8,    7,   15,   51,  404,
           49,  158,  139,  409,   92,  725, 2700,   14]])


배치 크기가 5이기 때문에 5개의 샘플이 출력됨. 각 샘플의 길이는 20의 길이를 가지는데, 이는 앞서 초기에 필드를 정의할 때 fix_length를 20으로 정해주었기 때문. 하나의 미니 배치의 크기는 (배치 크기 × fix_length)임.  

샘플의 중간, 중간에는 숫자 0이 존재하는데 이는 단어 집합에 포함되지 못한 단어들은 unk라는 토큰으로 변환되었음을 의미함. 또한 기존 샘플 길이가 20보다 작았던 샘플들은 뒤에 pad토큰의 번호인 숫자 1로 패딩됨

토치텍스트(TorchText)의 batch_first  
토치텍스트에서 배치퍼스트(batch_first)를 True로 한 경우와 False를 한 경우를 비교

In [23]:
import urllib.request
import pandas as pd

from torchtext.legacy import data
from torchtext.legacy.data import TabularDataset
from torchtext.legacy.data import Iterator

urllib.request.urlretrieve(
    "https://raw.githubusercontent.com/LawrenceDuan/IMDb-Review-Analysis/master/IMDb_Reviews.csv", 
    filename="IMDb_Reviews.csv")

df = pd.read_csv('IMDb_Reviews.csv', encoding='latin1')

train_df = df[:25000]
test_df = df[25000:]

train_df.to_csv("train_data.csv", index=False)
test_df.to_csv("test_data.csv", index=False)

# 필드 정의
# batch_first : 미니 배치 차원을 맨 앞으로 하여 데이터를 불러올 것인지 여부
TEXT = data.Field(
    sequential=True, use_vocab=True, tokenize=str.split,
    lower=True, batch_first=True, fix_length=20)
LABEL = data.Field(
    sequential=False, use_vocab=False, batch_first=False, is_target=True)

train_data, test_data = TabularDataset.splits(
        path='.', train='train_data.csv', test='test_data.csv', format='csv',
        fields=[('text', TEXT), ('label', LABEL)], skip_header=True)

# 정의한 필드에 build_vocab() 를 통해 10000개의 단어집합 생성
TEXT.build_vocab(train_data, min_freq=10, max_size=10000) 

# 첫번째 배치 출력
batch_size = 5
train_loader = Iterator(dataset=train_data, batch_size = batch_size)
batch = next(iter(train_loader)) # 첫번째 미니배치
print('batch.text : ', batch.text)
print('batch.text.shape : ', batch.text.shape)

batch.text :  tensor([[   0,    7,    3,   49,  838,   20,    6,    0,   43,    0,    6,   98,
            5,   11,   15,  386,    0,    5,    2,  501],
        [3013,    0,   10,  150, 2144, 1444,  354,   32,  949,    6,    2,  501,
            5,  104, 8700, 6905,    4,    0,   35,  224],
        [   9,  205,  141,   12,  273,   55, 2236,  147,  171, 2714,    5,    3,
          167,    5,    2,  125,  186,    0,    2, 1849],
        [  10,   20,    7, 1190, 2440,   81,  219,    0, 1303,   17, 2611,   30,
            4,  276,  195,   19,    3,  422, 1834,  143],
        [  10,   25, 6137,    5,  366,  909,    4, 2654,  390, 2167, 4653,    0,
            7,    3, 3739,  852, 1391,   16, 2035,    0]])
batch.text.shape :  torch.Size([5, 20])


batch_first = False로 하였을 경우의 텐서 크기

In [24]:
# 필드 정의
TEXT = data.Field(sequential=True, use_vocab=True, tokenize=str.split,
                  lower=True, fix_length=20)
LABEL = data.Field(
    sequential=False, use_vocab=False, batch_first=False, is_target=True)
train_data, test_data = TabularDataset.splits(
        path='.', train='train_data.csv', test='test_data.csv', format='csv',
        fields=[('text', TEXT), ('label', LABEL)], skip_header=True)

# 정의한 필드에 build_vocab() 를 통해 10000개의 단어집합 생성
TEXT.build_vocab(train_data, min_freq=10, max_size=10000) 

# 첫번째 배치 출력
batch_size = 5
train_loader = Iterator(dataset=train_data, batch_size = batch_size)

batch = next(iter(train_loader)) # 첫번째 미니배치

# 첫번째 미니 배치 출력
print('batch.text : ', batch.text)
# 하나의 미니 배치 크기는 (fix_length * 배치 크기)
print('batch.text.shape : ', batch.text.shape)

batch.text :  tensor([[   0, 3013,    9,   10,   10],
        [   7,    0,  205,   20,   25],
        [   3,   10,  141,    7, 6137],
        [  49,  150,   12, 1190,    5],
        [ 838, 2144,  273, 2440,  366],
        [  20, 1444,   55,   81,  909],
        [   6,  354, 2236,  219,    4],
        [   0,   32,  147,    0, 2654],
        [  43,  949,  171, 1303,  390],
        [   0,    6, 2714,   17, 2167],
        [   6,    2,    5, 2611, 4653],
        [  98,  501,    3,   30,    0],
        [   5,    5,  167,    4,    7],
        [  11,  104,    5,  276,    3],
        [  15, 8700,    2,  195, 3739],
        [ 386, 6905,  125,   19,  852],
        [   0,    4,  186,    3, 1391],
        [   5,    0,    0,  422,   16],
        [   2,   35,    2, 1834, 2035],
        [ 501,  224, 1849,  143,    0]])
batch.text.shape :  torch.Size([20, 5])
